# 🥗 Análisis de Adherencia a la Dieta

Este análisis evalúa si los socios cumplieron con la dieta asignada durante el período establecido. 
Se considera adherencia semanal cuando el socio asistió al gimnasio al menos 2 veces dentro de la semana correspondiente a su plan de dieta.

---


In [ ]:
# 📚 Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [ ]:
# 📥 Carga de datos
df_dietas = pd.read_csv("datasets/dieta_socio.csv")
df_asistencia = pd.read_csv("datasets/asistencia.csv")

# 🗓 Formatear fechas
df_dietas["fecha_inicio"] = pd.to_datetime(df_dietas["fecha_inicio"])
df_dietas["fecha_fin"] = df_dietas["fecha_inicio"] + pd.Timedelta(days=28)
df_asistencia["fecha"] = pd.to_datetime(df_asistencia["fecha"])


In [ ]:
# 📆 Calcular semana de asistencia
df_asistencia["semana"] = df_asistencia["fecha"].dt.to_period("W").apply(lambda r: r.start_time)


In [ ]:
# 📊 Agrupar asistencia semanal
asistencias_semanales = (
    df_asistencia.groupby(["socio_id", "semana"])
    .size()
    .reset_index(name="asistencias")
)


In [ ]:
# 🔄 Cruzar con fechas de dieta
adherencia = asistencias_semanales.merge(
    df_dietas[["socio_id", "fecha_inicio", "fecha_fin"]],
    on="socio_id"
)

# 📌 Filtrar semanas dentro del período de dieta
adherencia = adherencia[
    (adherencia["semana"] >= adherencia["fecha_inicio"]) &
    (adherencia["semana"] <= adherencia["fecha_fin"])
]


In [ ]:
# ✅ Marcar cumplimiento (al menos 2 asistencias por semana)
adherencia["cumple"] = adherencia["asistencias"] >= 2


In [ ]:
# 📈 Porcentaje de semanas cumplidas por socio
cumplimiento = (
    adherencia.groupby("socio_id")["cumple"]
    .mean()
    .reset_index(name="porcentaje_cumplimiento")
)

cumplimiento["porcentaje_cumplimiento"] = (cumplimiento["porcentaje_cumplimiento"] * 100).round(1)
cumplimiento.head()


In [ ]:
# 📊 Histograma de cumplimiento
sns.histplot(cumplimiento["porcentaje_cumplimiento"], bins=10, kde=True)
plt.title("Distribución de Adherencia a la Dieta")
plt.xlabel("% de semanas cumplidas")
plt.ylabel("Cantidad de socios")
plt.show()


## 🧾 Conclusiones

- Se evaluó la adherencia semanal de los socios a partir de su asistencia al gimnasio dentro del período de su dieta.
- Se consideró adherente al socio que asistió al menos 2 veces por semana.
- El gráfico muestra la distribución general del porcentaje de cumplimiento por socio.
- Este análisis permite detectar niveles bajos de adherencia y tomar medidas personalizadas para mejorar la constancia.

---
